# FM (SPECTRAL COMPOSITION)

Freqency Modulation (FM) is an audio algorithm that generates a wide variety of spectra, both harmonic and inharmonic, from just three input parameters: a carrier (center frequency), a carrier/modulator ratio, and an FM index controlling the density, or width, of the spectrum.  This demo is an example of spectral composition, in which a synthesis algorithm is used generate musical materials (chords, melodies) meant to be performed on traditional instruments.

<hr style="height:1px; color: grey;">

Notebook imports:

In [ ]:
import sys 
sys.path.append('/Users/taube/Software/musx')
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random
from musx import version, Score, Note, Seq, MidiFile, fmspectrum, keynum, hertz, odds, \
between, pick, setmidiplayer, playfile
print(f"musx version: {version}")

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")
print('OK!')

The `fm_chords()` composer generates a series of chords (fm spectra) using random fluctuations in C/M ratios and indexes to produce variations in chordal notes:

In [ ]:
def fm_chords(score, reps, cen, cm1, cm2, in1, in2, rhy):
    """
    Parameters
    ----------
    score : Score
        The scheduling queue to run the composer in.
    reps : int
        The number of chords the generator produces
    cen : int | float
        The fm carrier frequency 
    cm1 : int | float
        Lower bound c/m ratio.
    cm2 : int | float
        Upper bound c/m ratio.
    in1 : int | float
        Lower bound fm index.
    in2 : int | float
        Upper bound fm index.
    rhy : int | float
        The rhythm of the chords.
    """
    for _ in reps:
        # create the spectrum for the current chord
        spec = fmspectrum(hertz(cen), between(cm1, cm2), between(in1, in2)) 
        # convert spec to floating point keynums and force the pitches
        # to lie between minpitch and maxpitch.
        for k in spec.keynums(minpitch=48, maxpitch=72):
            m = Note(time=score.now, duration=rhy, pitch=k, amplitude=.5)
            score.add(m)
    # return the rhythmic wait time until this generator runs again
    yield rhy

print(f"fm_chords: {fm_chords}")

A list of key numbers that define the 'center frequency' for each fm spectrum.

In [ ]:
contour = keynum("a4 g f e a4 b c d gs b c5 ef fs g a5 bf g f e a5 b c d \
                  gs3 f e cs c bf5 gs5 as3 cs5 e6 f4 gs5 d6 e f g c5 b a \
                  g bf c5 cs e4 f gs d4 c b a4 e5 f g a5")
print(f"contour: {contour}")

`fm_impov()` is passed a contour line of carrier frequencies (specified as midi keynums) and generates fm spectra for melodic and harmonic gestures. To see the inputs and outputs of the spectra process remove the comments from the two print statements.

In [ ]:
def fm_improv(score, line, beat):
    """
    Parameters
    ----------
    score : Score
        The scheduling queue to run the composer in.
    line : list
        The contour line of pitches (floating point key numbers) that
        define the center note of each chord.
    beat : int | float
        The time to wait between chords.
    """
    amp = .7
    dur = beat
    for knum in line:
        ismel = odds(.7)
        rhy = pick(dur, dur / 2, dur / 4)
        f, r, x = hertz(knum), between(1.1, 1.9), pick(1, 2, 3)
        #print("\ncarrier=",f,"c/m ratio=",r,"fm index=",x)
        spec = fmspectrum(f,r,x)
        keys = spec.keynums(unique=True, minkey=knum-14, maxkey=knum+14)

        if ismel:
            random.shuffle(keys)
        sub = rhy / len(keys) if ismel else 0
        #print("melody:" if ismel else "chord:", "time=", score.now, "duration=", rhy, "keys=", keys)
        for i, k in enumerate(keys):
            m = Note(time=score.now + (i * sub), duration=dur, pitch=k, amplitude=amp)
            score.add(m)
        yield rhy

print(f"fm_improv: {fm_improv}")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack()
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1: {track1}")

Create a score and pass it track1 to hold the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Generate the composition:

In [ ]:
score.compose(fm_improv(score, contour, 1))
print("OK!")

Write the tracks to a midi file in the current directory:

In [ ]:
file = MidiFile("fm.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")

Play the output midi file if a terminal based midi player is installed:

In [ ]:
playfile(file.pathname)